# Experimenting Around

## First Trades

In [1]:
import os
import asyncio
import threading
from multiprocessing import Process
import numpy as np
import pandas as pd
from alpaca.trading.client import TradingClient
from alpaca.trading.enums import OrderSide, TimeInForce
from alpaca.trading.requests import MarketOrderRequest
from alpaca.trading.stream import TradingStream
import nest_asyncio # used for running event loops within Jupyter's event loop
import math

In [2]:
nest_asyncio.apply() # allows event loops to be run within jupyter

api_key = os.getenv("Public_Key")
secret_key = os.getenv("Secret")
client = TradingClient(api_key,secret_key,paper=True)
account = dict(client.get_account())
for k,v in account.items():
    print(f"{k:30}{v}")



id                            54404f7b-795a-4436-a230-7eb13b70da1b
account_number                PA3A0K6YRNTJ
status                        AccountStatus.ACTIVE
crypto_status                 AccountStatus.ACTIVE
currency                      USD
buying_power                  5875.26
regt_buying_power             5875.26
daytrading_buying_power       0
non_marginable_buying_power   937.63
cash                          87103.15
accrued_fees                  0
pending_transfer_out          None
pending_transfer_in           None
portfolio_value               99610.15
pattern_day_trader            False
trading_blocked               False
transfers_blocked             False
account_blocked               False
created_at                    2024-11-29 16:10:55.429486+00:00
trade_suspended_by_user       False
multiplier                    2
shorting_enabled              True
equity                        99610.15
last_equity                   99732.15
long_market_value             12507
short

In [ ]:
client.get

In [3]:
order_data = MarketOrderRequest(
    symbol = "SPY",
    qty = 100,
    side = OrderSide.BUY,
    time_in_force = TimeInForce.DAY
)
    
order = client.submit_order(order_data = order_data) # places the order


trades = TradingStream(api_key,secret_key,paper=True) # Websocket client
print(type(trades))
async def trade_status(data):
    print(data)
async def main():
    trades.subscribe_trade_updates(trade_status)
    trades.run()  # Await to handle the asynchronous nature properly

# await main()


APIError: {"buying_power":"5875.26","code":40310000,"cost_basis":"60103","message":"insufficient buying power"}

In [4]:
assets = [asset for asset in client.get_all_positions()]
positions = [(asset.symbol, asset.qty, asset.current_price) for asset in assets]
print("Postions")
print(f"{'Symbol':9}{'Qty':>4}{'Value':>15}")
print("-" * 28)
for position in positions:
    print(f"{position[0]:9}{position[1]:>4}{float(position[1]) * float(position[2]):>15.2f}")

client.close_all_positions(cancel_orders=True)
print()

Postions
Symbol    Qty          Value
----------------------------
AMD       100       12511.00



## Trying Some Actual Strategies/Visualizations

In [ ]:
!where python  


c:\Users\kaili\Desktop\Coding Projects\trading-bot\venv\Scripts\python.exe
C:\Users\kaili\AppData\Local\Programs\Python\Python312\python.exe


In [ ]:
import quantstats as qs
import yfinance

stock = "MCD"
portfolio = qs.utils.download_returns(stock,period='5y')    
print([f for f in dir(qs.stats) if f[0]!="_"])

['adjusted_sortino', 'autocorr_penalty', 'avg_loss', 'avg_return', 'avg_win', 'best', 'cagr', 'calmar', 'common_sense_ratio', 'comp', 'compare', 'compsum', 'conditional_value_at_risk', 'consecutive_losses', 'consecutive_wins', 'cpc_index', 'cvar', 'distribution', 'drawdown_details', 'expected_return', 'expected_shortfall', 'exposure', 'gain_to_pain_ratio', 'geometric_mean', 'ghpr', 'greeks', 'implied_volatility', 'information_ratio', 'kelly_criterion', 'kurtosis', 'max_drawdown', 'monthly_returns', 'omega', 'outlier_loss_ratio', 'outlier_win_ratio', 'outliers', 'payoff_ratio', 'pct_rank', 'probabilistic_adjusted_sortino_ratio', 'probabilistic_ratio', 'probabilistic_sharpe_ratio', 'probabilistic_sortino_ratio', 'profit_factor', 'profit_ratio', 'r2', 'r_squared', 'rar', 'recovery_factor', 'remove_outliers', 'risk_of_ruin', 'risk_return_ratio', 'rolling_greeks', 'rolling_sharpe', 'rolling_sortino', 'rolling_volatility', 'ror', 'serenity_index', 'sharpe', 'skew', 'smart_sharpe', 'smart_sor

# Testing Out Some Industries

In [3]:
df = pd.read_csv("pca-analysis-testing.csv")
df.head()

,Tech Giants,Social Media,Streaming & Entertainment,E-Commerce
0,AAPL,META,NFLX,AMZN
1,MSFT,SNAP,DIS,EBAY
2,GOOGL,PINS,AMZN,BABA
3,AMZN,TCEHY,WBD,JD
4,META,WB,CMCSA,SHOP


In [8]:
import yfinance as yf
import pandas as pd

# Define stock tickers for each industry
industries = {
    "Tech Giants": ["AAPL", "MSFT", "GOOGL", "AMZN", "META", "IBM", "ORCL", "SAP", "CRM", "ADBE", "ZM", "NOW", "WDAY", "PLTR", "SNOW", "DELL", "HPE", "INTU", "CSCO", "VMW"],
    "Social Media": ["META", "SNAP", "PINS", "TCEHY", "WB", "4689.T", "035720.KQ"],
    "Streaming & Entertainment": ["NFLX", "DIS", "AMZN", "WBD", "CMCSA", "PARA", "SPOT", "GOOGL", "ROKU", "VMEO", "TCEHY", "IQ", "BILI", "AAPL", "ZEEL", "VIAC", "SONY"],
    "E-Commerce": ["AMZN", "EBAY", "BABA", "JD", "SHOP", "WMT", "ETSY", "W", "RKUNY", "MELI", "TGT", "COST", "PDD", "CHWY", "OSTK", "REAL"]
}

# Function to download stock data and compute correlations
def compute_correlations(tickers):
    data = yf.download(tickers, start="2020-01-01", end="2024-12-01")
    return data.corr()

# Compute and store correlations for each industry
correlation_tables = {}
for industry, tickers in industries.items():
    print(f"Downloading data for {industry}...")
    correlation_tables[industry] = compute_correlations(tickers)

# Display correlation tables
for industry, correlation_table in correlation_tables.items():
    print(f"\nCorrelation Table for {industry}:\n")
    print(correlation_table)


[                       0%                       ]

[*********************100%***********************]  20 of 20 completed

1 Failed download:
['VMW']: YFTzMissingError('$%ticker%: possibly delisted; no timezone found')
[                       0%                       ]

[*********************100%***********************]  7 of 7 completed
[                       0%                       ]

[*********************100%***********************]  17 of 17 completed

2 Failed downloads:
['ZEEL', 'VIAC']: YFTzMissingError('$%ticker%: possibly delisted; no timezone found')
[******                12%                       ]  2 of 16 completed

[*********************100%***********************]  16 of 16 completed

1 Failed download:
['OSTK']: YFPricesMissingError('$%ticker%: possibly delisted; no price data found  (1d 2020-01-01 -> 2024-12-01)')



Correlation Table for Tech Giants:

Price            Adj Close     Close                                          \
Ticker                 VMW      AAPL      ADBE      AMZN       CRM      CSCO   
Price     Ticker                                                               
Adj Close VMW          NaN       NaN       NaN       NaN       NaN       NaN   
Close     AAPL         NaN  1.000000  0.460834  0.471489  0.515630  0.716741   
          ADBE         NaN  0.460834  1.000000  0.756975  0.801874  0.575335   
          AMZN         NaN  0.471489  0.756975  1.000000  0.863902  0.405012   
          CRM          NaN  0.515630  0.801874  0.863902  1.000000  0.492330   
...                    ...       ...       ...       ...       ...       ...   
Volume    SAP          NaN -0.063304 -0.266279 -0.194367 -0.223996 -0.232472   
          SNOW         NaN  0.147311 -0.136692 -0.047509 -0.053587 -0.012389   
          VMW          NaN       NaN       NaN       NaN       NaN       NaN   
   